https://adventofcode.com/2023/day/24

In [1]:
import re
from itertools import combinations

import numpy as np
import numpy.linalg as la

In [2]:
with open("data/24.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3
"""

In [4]:
def parse_puzzle(puzzle):
    L = []
    for line in puzzle.splitlines():
        nstrs = (x for x in re.split(r"\s+|@|,", line) if x)
        L.append(tuple(int(x) for x in nstrs))
    return L

In [5]:
def to_standard_form(x, y, vx, vy):
    m = vy / vx
    return np.array([m, -1, y - m * x])


def solve_sfs(*args):
    stacked = np.vstack(args)
    a = stacked[:, :-1]
    b = stacked[:, -1] * -1
    try:
        return la.solve(a, b)
    except la.LinAlgError as e:
        if e.args[0] == "Singular matrix":
            return None
        else:
            raise


class Hailstone:
    def __init__(self, pvs, minpos=200000000000000, maxpos=400000000000000):
        px, py, pz, vx, vy, vz = pvs
        self.ps = [px, py, pz]
        self.vs = [vx, vy, vz]
        self.point2d = (px, py)
        self.minpos = minpos
        self.maxpos = maxpos
        self.xy_std_form = to_standard_form(px, py, vx, vy)

    def intersection(self, other):
        return solve_sfs(self.xy_std_form, other.xy_std_form)

    def inbounds(self, point):
        x, y = point
        px, py = self.ps[:2]
        vx, vy = self.vs[:2]
        if not (self.minpos <= x <= self.maxpos and self.minpos <= y <= self.maxpos):
            return False
        if vx > 0 and x < px:
            return False
        if vx < 0 and x > px:
            return False
        if vy > 0 and y < py:
            return False
        if vy < 0 and y > py:
            return False
        return True

    def legit_intersection(self, other):
        point = self.intersection(other)
        return point is not None and self.inbounds(point) and other.inbounds(point)

    def __repr__(self):
        return "Hailstone((" + ", ".join(str(x) for x in self.ps + self.vs) + "))"

In [6]:
sum(
    a.legit_intersection(b)
    for a, b in combinations((Hailstone(x, 7, 27) for x in parse_puzzle(testdata)), 2)
)

2

In [7]:
%%time
sum(
    a.legit_intersection(b)
    for a, b in combinations((Hailstone(x) for x in parse_puzzle(data)), 2)
)

CPU times: user 344 ms, sys: 50.5 ms, total: 394 ms
Wall time: 329 ms


29142

### Part 2
Wow.

In [8]:
parse_puzzle(testdata)

[(19, 13, 30, -2, 1, -2),
 (18, 19, 22, -1, -1, -2),
 (20, 25, 34, -2, -2, -4),
 (12, 31, 28, -1, -2, -1),
 (20, 19, 15, 1, -5, -3)]

In [207]:
def throw_the_rock(puzzle):
    px = [x[0] for x in puzzle[:5]]
    py = [x[1] for x in puzzle[:5]]
    pz = [x[2] for x in puzzle[:5]]
    vx = [x[3] for x in puzzle[:5]]
    vy = [x[4] for x in puzzle[:5]]
    vz = [x[5] for x in puzzle[:5]]
    A = np.zeros((4, 4), dtype=np.longlong)
    b = np.zeros(4, dtype=np.longlong)
    for (aind, i, j) in [(0, 0, 1), (1, 0, 2), (2, 0, 3), (3, 0, 4)]:
        A[aind] = [vy[i] - vy[j], py[j] - py[i], vx[j] - vx[i], px[i] - px[j]]
        b[aind] = px[i] * vy[i] - py[i] * vx[i] - px[j] * vy[j] + py[j] * vx[j]
    rpx, rvx, rpy, rvy = np.round(la.solve(A, b)).astype(int)
    i, j = 1, 2
    ti = (px[i] - rpx) / (rvx - vx[i])
    tj = (px[j] - rpx) / (rvx - vx[j])
    C = np.array([
        [1, ti],
        [1, tj]
    ], dtype=np.longlong)
    d = np.array([pz[i] + vz[i] * ti, pz[j] + vz[j] * tj], dtype=np.longlong)
    rpz, rvz = np.round(la.solve(C, d)).astype(int)
    return rpx, rvx, rpy, rvy, rpz, rvz, i, j, ti, tj

In [177]:
throw_the_rock(parse_puzzle(testdata))

(24, -3, 13, 1, 10, 2, 1, 2, 3.0, 4.0)

In [194]:
puzzle = parse_puzzle(data)

In [195]:
rpx, rvx, rpy, rvy, rpz, rvz, i, j, ti, tj = throw_the_rock(puzzle)
rpx, rvx, rpy, rvy, rpz, rvz, i, j, ti, tj

(334948624416533,
 -86,
 371647004954418,
 -143,
 142351957892081,
 289,
 1,
 2,
 547833135830.0,
 728788861354.0)

848947587263032 is too low.

In [196]:
ti

547833135830.0

In [197]:
puzzle[i]

(310843966440013, 307550528062309, 305058399233591, -42, -26, -8)

In [198]:
puzzle[i][0] + puzzle[i][3] * ti

287834974735153.0

In [199]:
rpx + rvx * ti

287834974735153.0

In [200]:
puzzle[i][1] + puzzle[i][4] * ti

293306866530729.0

In [202]:
(rpy+1) + rvy * ti

293306866530729.0

In [203]:
puzzle[i][2] + puzzle[i][5] * ti

300675734146951.0

In [204]:
rpz + rvz * ti

300675734146951.0

In [206]:
rpx + rpy + rpz + 1

848947587263033